In [ ]:
! pip install nest_asyncio asyncio aiohttp

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio
import aiohttp
import json
import re
import ast

# ---------------------------
# Configuration Constants
# ---------------------------
OPENROUTER_API_KEY = "sk-xxxx" # Replace with your OpenRouter API key
JINA_API_KEY = "jina_xxxx" # Replace with your Jina API key

OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
JINA_BASE_URL = "https://r.jina.ai/"

DEFAULT_MODEL = "anthropic/claude-3.5-haiku"
REASON_MODEL = "deepseek/deepseek-r1-distill-qwen-32b"

BASE_SEARXNG_URL = "http://localhost:4000/search"

In [ ]:
# ============================
# Asynchronous Helper Functions
# ============================

async def call_openrouter_async(session, messages, model=DEFAULT_MODEL):
    """
    Asynchronously call the OpenRouter chat completion API with the provided messages.
    Returns the content of the assistant’s reply.
    """
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "X-Title": "OpenDeepResearcher, by Matt Shumer and Benhao Tang",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": messages
    }
    try:
        async with session.post(OPENROUTER_URL, headers=headers, json=payload) as resp:
            if resp.status == 200:
                result = await resp.json()
                try:
                    return result['choices'][0]['message']['content']
                except (KeyError, IndexError) as e:
                    print("Unexpected OpenRouter response structure:", result)
                    return None
            else:
                text = await resp.text()
                print(f"OpenRouter API error: {resp.status} - {text}")
                return None
    except Exception as e:
        print("Error calling OpenRouter:", e)
        return None

async def make_initial_searching_plan_async(session, user_query):
    """
    Ask the reasoning LLMs to produce a research plan based on the user’s query.
    """
    prompt = (
        "You are an advanced reasoning LLM that specializes in structuring and refining research plans. Based on the given user query,"
        "you will generate a comprehensive research plan that expands on the topic, identifies key areas of investigation, and breaks down"
        " the research process into actionable steps for a search agent to execute.\n"
        "Process:\n"
        "Expand the Query: 1. Clarify and enrich the user’s query by considering related aspects, possible interpretations,"
        "and necessary contextual details. 2.Identify any ambiguities and resolve them by assuming the most logical and useful framing of the problem.\n"
        "Identify Key Research Areas: 1. Break down the expanded query into core themes, subtopics, or dimensions of investigation."
        "2.Determine what information is necessary to provide a comprehensive answer.\n"
        "Define Research Steps: 1. Outline a structured plan with clear steps that guide the search agent on how to gather information."
        "2. Specify which sources or types of data are most relevant (e.g., academic papers, government reports, news sources, expert opinions)."
        "3. Prioritize steps based on importance and logical sequence.\n"
        "Suggest Search Strategies: 1.Recommend search terms, keywords, and boolean operators to optimize search efficiency."
        "2. Identify useful databases, journals, and sources where high-quality information can be found."
        "3. Suggest methodologies for verifying credibility and synthesizing findings.\n"
        "NO EXPLANATIONS, write plans ONLY!\n"
    )
    messages = [
        {"role": "system", "content": "You are an advanced reasoning LLM that guides a following search agent to search for relevant information for a research."},
        {"role": "user", "content": f"User Query: {user_query}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages, model=REASON_MODEL)
    if response:
        try:
            # Remove <think>...</think> tags and their content if they exist
            cleaned_response = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL).strip()
            print(f"Plan:{cleaned_response}")
            return cleaned_response
        except Exception as e:
            print(f"Error processing response: {e}")
    return []

async def judge_search_result_and_future_plan_aync(session, user_query, original_plan, context_combined):
    """
    Ask the reasoning LLMs to judge the result of the search attempt and produce a plan for next interation.
    """
    prompt = (
    "You are an advanced reasoning LLM that specializes in evaluating research results and refining search strategies. "
    "Your task is to analyze the search agent’s findings, assess their relevance and completeness, "
    "and generate a structured plan for the next search iteration. Your goal is to ensure a thorough and efficient research process "
    "that ultimately provides a comprehensive answer to the user’s query. But still, if you think everything is enough, you can tell search agent to stop\n"
    "Process:\n"
    "1. **Evaluate Search Results:**\n"
    "   - Analyze the retrieved search results to determine their relevance, credibility, and completeness.\n"
    "   - Identify missing information, knowledge gaps, or weak sources.\n"
    "   - Assess whether the search results sufficiently address the key research areas from the original plan.\n"
    "   - If everything is enough, tell search agent to stop with your reason\n"
    "2. **Determine Next Steps:**\n"
    "   - Based on gaps identified, refine or expand the research focus.\n"
    "   - Suggest additional search directions or alternative sources to explore.\n"
    "   - If necessary, propose adjustments to search strategies, including keyword modifications, new sources, or filtering techniques.\n"
    "3. **Generate an Updated Research Plan:**\n"
    "   - Provide a structured step-by-step plan for the next search iteration.\n"
    "   - Clearly outline what aspects need further investigation and where the search agent should focus next.\n"
    "NO EXPLANATIONS, write plans ONLY!\n"
    "Now, based on the above information and instruction, evaluate the search results and generate a refined research plan for the next iteration."
    )

    messages = [
        {"role": "system", "content": "You are an advanced reasoning LLM that guides a following search agent to search for relevant information for a research."},
        {"role": "user", "content": f"User Query: {user_query}\n\n Original Research Plan: {original_plan} \n\nExtracted Relevant Contexts by the search agent:\n{context_combined} \n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages, model=REASON_MODEL)
    if response:
        try:
            # Remove <think>...</think> tags and their content if they exist
            cleaned_response = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL).strip()
            print(f"Next Plan:{cleaned_response}")
            return cleaned_response
        except Exception as e:
            print(f"Error processing response: {e}")
    return []

async def generate_writing_plan_aync(session, user_query, aggregated_contexts):
    """
    Ask the reasoning LLM to generate a structured writing plan for the final report based on the aggregated research findings.
    """
    prompt = (
        "You are an advanced reasoning LLM that specializes in structuring comprehensive research reports. "
        "Your task is to analyze the aggregated research findings from previous searches and generate a structured plan "
        "for writing the final report. The goal is to ensure that the report is well-organized, complete, and effectively presents the findings.\n\n"
        "Process:\n"
        "1. **Analyze Aggregated Findings:**\n"
        "   - Review the provided research findings for relevance, accuracy, and coherence.\n"
        "   - Identify key insights, supporting evidence, and significant conclusions.\n"
        "   - Highlight any inconsistencies or areas that may need further clarification.\n\n"
        "2. **Determine Report Structure:**\n"
        "   - Define a clear outline with logical sections that effectively communicate the research results.\n"
        "   - Ensure the structure follows a coherent flow, such as Introduction, Key Findings, Analysis, Conclusion, and References.\n"
        "   - Specify where different pieces of evidence should be integrated within the report.\n\n"
        "3. **Provide Writing Guidelines:**\n"
        "   - Suggest the tone and style appropriate for the report (e.g., formal, analytical, concise).\n"
        "   - Recommend how to synthesize multiple sources into a coherent narrative.\n"
        "   - If any section lacks sufficient information, indicate where additional elaboration may be needed.\n\n"
        "NO EXPLANATIONS, write plans ONLY!\n"
        "Now, based on the above instructions, generate a structured plan for writing the final research report.\n"
    )

    messages = [
        {"role": "system", "content": "You are an advanced reasoning LLM that structures research findings into a well-organized final report."},
        {"role": "user", "content": f"User Query: {user_query}\n\nAggregated Research Findings:\n{aggregated_contexts}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages, model=REASON_MODEL)
    if response:
        try:
            # Remove <think>...</think> tags and their content if they exist
            cleaned_response = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL).strip()
            print(f"Writing Plan: {cleaned_response}")
            return cleaned_response
        except Exception as e:
            print(f"Error processing response: {e}")
    return []

    
async def generate_search_queries_async(session, query_plan):
    """
    Ask the LLM to produce up to four precise search queries (in Python list format)
    based on the user’s query.
    """
    prompt = (
        "You are an expert research assistant. Given the user's query, generate up to four distinct, "
        "precise search queries that would help gather comprehensive information on the topic. "
        "Return only a Python list of strings, for example: ['query1', 'query2', 'query3']."
    )
    messages = [
        {"role": "system", "content": "You are a helpful and precise research assistant."},
        {"role": "user", "content": f"{query_plan}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages)
    if response:
        cleaned = response.strip()
        
        # Remove triple backticks and language specifier if present
        cleaned = re.sub(r"```(?:\w+)?\n(.*?)\n```", r"\1", cleaned, flags=re.DOTALL).strip()

        # First, try to directly evaluate the cleaned string
        try:
            new_queries = ast.literal_eval(cleaned)
            if isinstance(new_queries, list):
                return new_queries
        except Exception as e:
            # Direct evaluation failed; try to extract the list part from the string.
            match = re.search(r'(\[.*\])', cleaned, re.DOTALL)
            if match:
                list_str = match.group(1)
                try:
                    new_queries = ast.literal_eval(list_str)
                    if isinstance(new_queries, list):
                        return new_queries
                except Exception as e_inner:
                    print("Error parsing extracted list:", e_inner, "\nExtracted text:", list_str)
                    return []
            print("Error parsing new search queries:", e, "\nResponse:", response)
            return []
    return []


async def perform_search_async(session, query):
    params = {
        "q": query,
        "format": "json"
    }
    try:
        async with session.get(BASE_SEARXNG_URL, params=params) as resp:
            if resp.status == 200:
                results = await resp.json()
                if "results" in results:
                    # Extract the URLs from the returned results list.
                    links = [item.get("url") for item in results["results"] if "url" in item]
                    return links
                else:
                    print("No results in SearXNG response.")
                    return []
            else:
                text = await resp.text()
                print(f"SearXNG error: {resp.status} - {text}")
                return []
    except Exception as e:
        print("Error performing SearXNG search:", e)
        return []


async def fetch_webpage_text_async(session, url):
    """
    Asynchronously retrieve the text content of a webpage using Jina.
    The URL is appended to the Jina endpoint.
    """
    full_url = f"{JINA_BASE_URL}{url}"
    headers = {
        "Authorization": f"Bearer {JINA_API_KEY}"
    }
    try:
        async with session.get(full_url, headers=headers) as resp:
            if resp.status == 200:
                return await resp.text()
            else:
                text = await resp.text()
                print(f"Jina fetch error for {url}: {resp.status} - {text}")
                return ""
    except Exception as e:
        print("Error fetching webpage text with Jina:", e)
        return ""


async def is_page_useful_async(session, user_query, page_text, page_url):
    """
    Ask the LLM if the provided webpage content is useful for answering the user's query.
    The LLM must reply with exactly "Yes" or "No".
    """
    prompt = (
        "You are a critical research evaluator. Given the user's query and the content of a webpage, "
        "determine if the webpage contains information relevant and useful for addressing the query. "
        "Respond with exactly one word: 'Yes' if the page is useful, or 'No' if it is not. Do not include any extra text."
    )
    messages = [
        {"role": "system", "content": "You are a strict and concise evaluator of research relevance."},
        {"role": "user", "content": f"User Query: {user_query}\n\nWeb URL: {page_url}\n\nWebpage Content (first 20000 characters):\n{page_text[:20000]}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages)
    if response:
        answer = response.strip()
        if answer in ["Yes", "No"]:
            return answer
        else:
            # Fallback: try to extract Yes/No from the response.
            if "Yes" in answer:
                return "Yes"
            elif "No" in answer:
                return "No"
    return "No"


async def extract_relevant_context_async(session, user_query, search_query, page_text, page_url):
    """
    Given the original query, the search query used, and the page content,
    have the LLM extract all information relevant for answering the query.
    """
    prompt = (
        "You are an expert information extractor. Given the user's query, the search query that led to this page, "
        "and the webpage content, extract all pieces of information that are relevant to answering the user's query. "
        "Return only the relevant context as plain text without commentary."
    )
    messages = [
        {"role": "system", "content": "You are an expert in extracting and summarizing relevant information."},
        {"role": "user", "content": f"User Query: {user_query}\nSearch Query: {search_query}\n\nWeb URL: {page_url}\n\nWebpage Content (first 20000 characters):\n{page_text[:20000]}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages)
    if response:
        return response.strip()
    return ""


async def get_new_search_queries_async(session, user_query, new_research_plan, previous_search_queries, all_contexts):
    """
    Based on the original query, the previously used search queries, and all the extracted contexts,
    ask the LLM whether additional search queries are needed. If yes, return a Python list of up to four queries;
    if the LLM thinks research is complete, it should return "<done>".
    """
    context_combined = "\n".join(all_contexts)
    prompt = (
        "You are an analytical research assistant. Based on the original query, the search queries performed so far, "
        "the next step plan by a planning agent and the extracted contexts from webpages, determine if further research is needed. "
        "If further research is needed, ONLY provide up to four new search queries as a Python list (for example, "
        "['new query1', 'new query2']). If you believe no further research is needed, respond with exactly <done>."
        "\nREMEMBER: Output ONLY a Python list or the token <done> WITHOUT any additional text or explanations."
    )
    messages = [
        {"role": "system", "content": "You are a systematic research planner."},
        {"role": "user", "content": f"User Query: {user_query}\nPrevious Search Queries: {previous_search_queries}\n\nExtracted Relevant Contexts:\n{context_combined}Next Research Plan by planning agent:{new_research_plan}\n\n{prompt}"}
    ]
    response = await call_openrouter_async(session, messages)
    if response:
        cleaned = response.strip()
        if cleaned == "<done>":
            return "<done>"
        
        # Remove code env of ``` if present
        cleaned = re.sub(r"```(?:\w+)?\n(.*?)\n```", r"\1", cleaned, flags=re.DOTALL).strip()

        # First, try to directly evaluate the cleaned string
        try:
            new_queries = ast.literal_eval(cleaned)
            if isinstance(new_queries, list):
                return new_queries
        except Exception as e:
            # Direct evaluation failed; try to extract the list part from the string.
            match = re.search(r'(\[.*\])', cleaned, re.DOTALL)
            if match:
                list_str = match.group(1)
                try:
                    new_queries = ast.literal_eval(list_str)
                    if isinstance(new_queries, list):
                        return new_queries
                except Exception as e_inner:
                    print("Error parsing extracted list:", e_inner, "\nExtracted text:", list_str)
                    return []
            print("Error parsing new search queries:", e, "\nResponse:", response)
            return []
    return []


async def generate_final_report_async(session, user_query, report_planning, all_contexts):
    """
    Generate the final comprehensive report using all gathered contexts.
    """
    context_combined = "\n".join(all_contexts)
    prompt = (
        "You are an expert researcher and report writer. Based on the gathered contexts below and the original query, "
        "write a comprehensive, well-structured, and detailed report that addresses the query thoroughly. "
        "Include all relevant insights and conclusions without extraneous commentary."
        "Math equations should use proper LaTeX syntax in markdown format, with $\\LaTeX{}$ for inline, $$\\LaTeX{}$$ for block."
        "Properly cite all the sources inline from 'Gathered Relevant Contexts' with [cite_number],"
        "and a corresponding bibliography list with their urls in markdown format in the end."
    )
    messages = [
        {"role": "system", "content": "You are a skilled report writer."},
        {"role": "user", "content": f"User Query: {user_query}\n\nGathered Relevant Contexts:\n{context_combined}\n\nWriting plan from a planning agent:\n{report_planning}\n\nWriting Instructions:{prompt}"}
    ]
    report = await call_openrouter_async(session, messages)
    return report


async def process_link(session, link, user_query, search_query):
    """
    Process a single link: fetch its content, judge its usefulness, and if useful, extract the relevant context.
    """
    print(f"Fetching content from: {link}")
    page_text = await fetch_webpage_text_async(session, link)
    if not page_text:
        return None
    usefulness = await is_page_useful_async(session, user_query, page_text, link)
    print(f"Page usefulness for {link}: {usefulness}")
    if usefulness == "Yes":
        context = await extract_relevant_context_async(session, user_query, search_query, page_text, link)
        if context:
            print(f"Extracted context from {link} (first 200 chars): {context[:200]}")
            context="url:"+link+"\ncontext:"+context
            return context
    return None


# =========================
# Main Asynchronous Routine
# =========================

async def async_main():
    user_query = input("Enter your research query/topic: ").strip()
    iter_limit_input = input("Enter maximum number of iterations (default 10): ").strip()
    iteration_limit = int(iter_limit_input) if iter_limit_input.isdigit() else 10

    aggregated_contexts = []    # All useful contexts from every iteration
    all_search_queries = []     # Every search query used across iterations
    iteration = 0

    async with aiohttp.ClientSession() as session:
        # ----- INITIAL SEARCH QUERIES -----
        reseach_plan = await make_initial_searching_plan_async(session, user_query)
        initial_query = "User Query:" + user_query + "\n\nResaerch Plan:" + reseach_plan
        new_search_queries = await generate_search_queries_async(session, initial_query)
        if not new_search_queries:
            print("No search queries were generated by the LLM. Exiting.")
            return
        all_search_queries.extend(new_search_queries)

        # ----- ITERATIVE RESEARCH LOOP -----
        while iteration < iteration_limit:
            print(f"\n=== Iteration {iteration + 1} ===")
            iteration_contexts = []

            # For each search query, perform searxng searches concurrently.
            search_tasks = [perform_search_async(session, query) for query in new_search_queries]
            search_results = await asyncio.gather(*search_tasks)

            # Aggregate all unique links from all search queries of this iteration.
            # Map each unique link to the search query that produced it.
            unique_links = {}
            for idx, links in enumerate(search_results):
                query = new_search_queries[idx]
                for link in links:
                    if link not in unique_links:
                        unique_links[link] = query

            print(f"Aggregated {len(unique_links)} unique links from this iteration.")

            # Process each link concurrently: fetch, judge, and extract context.
            link_tasks = [
                process_link(session, link, user_query, unique_links[link])
                for link in unique_links
            ]
            link_results = await asyncio.gather(*link_tasks)

            # Collect non-None contexts.
            for res in link_results:
                if res:
                    iteration_contexts.append(res)

            if iteration_contexts:
                aggregated_contexts.extend(iteration_contexts)
            else:
                print("No useful contexts were found in this iteration.")

            # ----- ASK THE LLM IF MORE SEARCHES ARE NEEDED AND NOT THE FINIAL ROUND -----
            if iteration + 1 < iteration_limit:
                new_research_plan = await judge_search_result_and_future_plan_aync(session, user_query, reseach_plan, aggregated_contexts)
                new_search_queries = await get_new_search_queries_async(session, user_query, new_research_plan, all_search_queries, aggregated_contexts)
                if new_search_queries == "<done>":
                    print("LLM indicated that no further research is needed.")
                    break
                elif new_search_queries:
                    print("LLM provided new search queries:", new_search_queries)
                    all_search_queries.extend(new_search_queries)
                else:
                    print("LLM did not provide any new search queries. Ending the loop.")
                    break

            iteration += 1

        # ----- FINAL REPORT -----
        print("\nGenerating final report...")
        final_report_planning = await generate_writing_plan_aync(session, user_query, aggregated_contexts)
        final_report = await generate_final_report_async(session, user_query, final_report_planning, aggregated_contexts)
        print("\n==== FINAL REPORT ====\n")
        print(final_report)


def main():
    asyncio.run(async_main())

In [ ]:
if __name__ == "__main__":
    main()